In [1]:
#!pip install xstac cognito_client rio_tiler
#!pip install morecantile==3.4.0
#!pip install aiobotocore==2.5.0
#!pip install pydantic==1.10.10

In [2]:
from helpers import eodc_hub_role
import pystac
import s3fs
import xstac
from titiler_xarray.titiler.xarray.reader import xarray_open_dataset

In [3]:
dataset_url = 's3://veda-data-store-staging/cmip6-GISS-E2-1-G-tas-kerchunk/combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk.json'
ds = xarray_open_dataset(dataset_url, reference=True)

/home/jovyan/tile-benchmarking/titiler_xarray/titiler/xarray/reader.py:44: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)


In [4]:
spatial_extent_values = [ds.lon[0].values, ds.lat[0].values, ds.lon[-1].values, ds.lat[-1].values]
spatial_extent = list(map(int, spatial_extent_values))
_id = 'combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk_TEST'
zarr_asset = pystac.Asset(
    title='zarr',
    href=dataset_url,
    media_type='application/vnd+zarr',
    roles=['data'],
)
extent = pystac.Extent(
    spatial=pystac.SpatialExtent(bboxes=[spatial_extent]),
    temporal=pystac.TemporalExtent([[None, None]])
)

In [5]:
providers = [
    pystac.Provider(
        name="VEDA",
        roles=[pystac.ProviderRole.PRODUCER, pystac.ProviderRole.PROCESSOR, pystac.ProviderRole.HOST],
        url="https://github.com/nasa-impact/veda-data-pipelines",
    )
]

In [6]:
collection = pystac.Collection(
    id=_id,
    extent=extent,
    assets = {'zarr': zarr_asset},
    description='for zarr testing',
    providers=providers,
    stac_extensions=['https://stac-extensions.github.io/datacube/v2.0.0/schema.json'],
    license="CC0-1.0"
)

In [7]:
collection

<Collection id=combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk_TEST>

In [8]:
collection_template = collection.to_dict()
collection = xstac.xarray_to_stac(
    ds,
    collection_template,
    temporal_dimension="time",
    x_dimension="lon",
    y_dimension="lat",
    # TODO: get this from attributes if possible
    reference_system="4326",
    validate=False
)

In [9]:
collection.validate()

['https://schemas.stacspec.org/v1.0.0/collection-spec/json-schema/collection.json',
 'https://stac-extensions.github.io/datacube/v2.0.0/schema.json']

In [10]:
from cognito_client import CognitoClient

client = CognitoClient(
    client_id="795d7mgl4ndt2tjahr4hjnmt1m",
    user_pool_id="us-west-2_7frfmEM6F",
    identity_pool_id="us-west-2:f1b60666-87b0-4df7-bccd-48ba7f79b9bc",
)
_ = client.login()

TOKEN = client.access_token

Enter your Cognito username:  aimee
Enter your Cognito password:  ········


In [11]:
API = "https://6r8ht9b123.execute-api.us-west-2.amazonaws.com/dev/"

In [12]:
dataset = collection.to_dict()
dataset['data_type'] = 'zarr'
dataset['collection'] = _id
dataset['title'] = 'CMIP6 Daily GISS-E2-1-G TAS Kerchunk (TEST)'
dataset['dashboard:is_periodic'] = True
dataset['dashboard:time_density'] = 'day'

In [13]:
import requests
auth_header = f"Bearer {TOKEN}"
headers = {
    "Authorization": auth_header,
    "content-type": "application/json",
    "accept": "application/json",
}
response = requests.post((API + "collections"), json=dataset, headers=headers)
#response.raise_for_status()
print(response.text)


["Successfully published: combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk_TEST"]
